In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/f3497400bbf440cfb39abfd2d1f90440



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [2]:
# tensorflow 2.7 
import tensorflow as tf 
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization,Concatenate
from tensorflow.keras.models import Model, Sequential

In [3]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [4]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

In [5]:
df_train_features.head(1)

,trackID,title,tags,loudness,tempo,time_signature,key,mode,duration,vect_1,...,vect_139,vect_140,vect_141,vect_142,vect_143,vect_144,vect_145,vect_146,vect_147,vect_148
0,8424,The Calamity [Explicit],"the, you, to, a, not, in, of, that, are, we, w...",-2.824,131.969,4,0,1,298.13506,52.050795,...,0.001097,0.001161,0.001225,0.001251,0.001238,0.001202,0.00113,0.00104,0.104525,0.335818


In [6]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

num_train_meta_data = df_train_features.iloc[:,3:5]
num_valid_meta_data = df_valid_features.iloc[:,3:5]

num_train_meta_data['duration'] = df_train_features['duration']
num_valid_meta_data['duration'] = df_valid_features['duration']

In [7]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)

num_train_meta = scaler.fit_transform(num_train_meta_data)
num_valid_meta = scaler.fit_transform(num_valid_meta_data)

In [8]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

In [9]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
)

In [13]:
class AudioNumMetaClassifier:
    def __init__(self,num_train,num_meta_train,num_train_y,num_valid,num_valid_meta,num_valid_y,class_weight,early_stop,experiment):
        self.num_train = num_train
        self.num_meta_train = num_meta_train    
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_meta = num_valid_meta
        self.num_valid_y = num_valid_y
        self.class_weight = class_weight
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                units=self.experiment.get_parameter("audio_layer_1"),
                activation='elu',
                input_shape=(num_train.shape[1],)
            )
        )
        model.add(Dropout(0.5))
        model.add(BatchNormalization())

        model.add(
            Dense(
                units=self.experiment.get_parameter("audio_layer_2"),
                activation='elu'))
        model.add(Dropout(0.3))
        model.add(BatchNormalization())

        model_meta = Sequential()
        model_meta.add(
            Dense(
                units=self.experiment.get_parameter("meta_layer_1"),
                activation='elu',
                input_shape=(num_train_meta.shape[1],)
            )
        )
        model_meta.add(Dropout(0.5))
        model_meta.add(BatchNormalization())

        model_meta.add(
            Dense(
                units=self.experiment.get_parameter("meta_layer_2"),
                activation='elu'))
        model_meta.add(Dropout(0.3))
        model_meta.add(BatchNormalization())


        merged_out = Concatenate()([model.output,model_meta.output])
        merged_out = Dense(
            units=self.experiment.get_parameter("final_layer"),
            activation='elu')(merged_out)
        merged_out = Dense(8,activation='softmax')(merged_out)

        combined_model = Model([model.input] + [model_meta.input], [merged_out])
        combined_model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return combined_model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            [self.num_train,self.num_meta_train],
            self.num_train_y,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=([self.num_valid,self.num_valid_meta],self.num_valid_y),
            class_weight=self.class_weight,
            callbacks=[self.early_stop],
            shuffle=True
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate([self.num_valid,self.num_valid_meta],self.num_valid_y)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [14]:
classifier = AudioNumMetaClassifier(num_train,num_train_meta,num_train_y,num_valid,num_valid_meta,num_valid_y,class_weight,early_stop,experiment)

In [15]:
classifier.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "audio_layer_1": {"type": "discrete", "values": [400,450,500,550,600,700,900,1000]},
        "audio_layer_2": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "meta_layer_1": {"type": "discrete", "values": [300,400,450,500,550,600,700,900,1000]},
        "meta_layer_2": {"type": "discrete", "values": [128,256,300,400,450,500,550]},
        "final_layer": {"type": "discrete", "values": [20,32,64,128,256]},
        "batch_size": {"type": "discrete", "values": [16,32]},
        "epochs": {"type": "discrete", "values": [15,20,25,30]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=019075b278c649c7b9842acd5f3dcf6d
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 141120, 'endTime': None, 'id': '019075b278c649c7b9842acd5f3dcf6d', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'audio_layer_1': {'type': 'discrete', 'values': [400, 450, 500, 550, 600, 700, 900, 1000]}, 'audio_layer_2': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}, 'batch_size': {'type': 'discrete', 'values': [16, 32]}, 'epochs': {'type': 'discrete', 'values': [15, 20, 25, 30]}, 'final_layer': {'type': 'discrete', 'values': [20, 32, 64, 128, 256]}, 'meta_layer_1': {'type': 'discrete', 'values': [300, 400, 450, 500, 550, 600, 700, 900, 1000]}, 'meta_layer_2': {'type': 'discrete', 'values': [128, 256, 300, 400, 450, 500, 550]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLim

Epoch 1/15


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


240/240 [==============================] - 3s 6ms/step - loss: 1.6898 - accuracy: 0.4066 - val_loss: 1.5073 - val_accuracy: 0.4533
Epoch 2/15
240/240 [==============================] - 1s 5ms/step - loss: 1.3947 - accuracy: 0.4931 - val_loss: 1.5213 - val_accuracy: 0.4378
Epoch 3/15
240/240 [==============================] - 1s 5ms/step - loss: 1.3293 - accuracy: 0.5059 - val_loss: 1.3605 - val_accuracy: 0.5267
Epoch 4/15
240/240 [==============================] - 1s 5ms/step - loss: 1.2713 - accuracy: 0.5313 - val_loss: 1.3643 - val_accuracy: 0.5156
Epoch 5/15
240/240 [==============================] - 1s 5ms/step - loss: 1.2181 - accuracy: 0.5376 - val_loss: 1.4668 - val_accuracy: 0.4467
Epoch 6/15
240/240 [==============================] - 1s 5ms/step - loss: 1.1837 - accuracy: 0.5563 - val_loss: 1.3697 - val_accuracy: 0.4867
Epoch 7/15
240/240 [==============================] - 2s 8ms/step - loss: 1.1745 - accuracy: 0.5621 - val_loss: 1.3660 - val_accuracy: 0.5089
Epoch 8/15
240/24

COMET INFO: [1.2989434003829956, 0.5444444417953491]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/80076188e4d94fb3ac9eef9665f17b31
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [22]                : (0.40661630034446716, 0.6081010699272156)
COMET INFO:     batch_accuracy [528]         : (0.15625, 0.71875)
COMET INFO:     batch_loss [528]             : (0.7330235242843628, 2.6585617065429688)
COMET INFO:     epoch_duration [22]          : (1.1560000000026776, 4.76600000000326)
COMET INFO:     loss [22]                    : (1.0046277046203613, 1.689835548400879)
COMET INFO:     val_accuracy [22]            : (0.4377777874469757, 0.5444444417953491)
COMET INFO:     val_loss [22]                : (1.2989434003829956, 1.5

...

COMET INFO: [1.3414344787597656, 0.5400000214576721]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/a38ec5e9fb244a4cb89ddcaf04c3555e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [20]                : (0.415733277797699, 0.6229487061500549)
COMET INFO:     batch_accuracy [480]         : (0.0625, 0.78125)
COMET INFO:     batch_loss [480]             : (0.7190606594085693, 2.235048294067383)
COMET INFO:     epoch_duration [20]          : (1.2190000000118744, 2.812000000005355)
COMET INFO:     loss [20]                    : (0.9601150751113892, 1.6898738145828247)
COMET INFO:     val_accuracy [20]            : (0.41999998688697815, 0.5644444227218628)
COMET INFO:     val_loss [20]                : (1.3089936971664429, 1.64

Epoch 1/20
480/480 [==============================] - 4s 7ms/step - loss: 1.7140 - accuracy: 0.4082 - val_loss: 1.4569 - val_accuracy: 0.4400
Epoch 2/20
480/480 [==============================] - 2s 5ms/step - loss: 1.4450 - accuracy: 0.4667 - val_loss: 1.4453 - val_accuracy: 0.4933
Epoch 3/20
480/480 [==============================] - 2s 5ms/step - loss: 1.3556 - accuracy: 0.4905 - val_loss: 1.5315 - val_accuracy: 0.4556
Epoch 4/20
480/480 [==============================] - 3s 6ms/step - loss: 1.2984 - accuracy: 0.5121 - val_loss: 1.4435 - val_accuracy: 0.4644
Epoch 5/20
480/480 [==============================] - 3s 6ms/step - loss: 1.2606 - accuracy: 0.5267 - val_loss: 1.5037 - val_accuracy: 0.4733
Epoch 6/20
480/480 [==============================] - 2s 5ms/step - loss: 1.2298 - accuracy: 0.5371 - val_loss: 1.5495 - val_accuracy: 0.4622
Epoch 7/20
480/480 [==============================] - 2s 5ms/step - loss: 1.2011 - accuracy: 0.5447 - val_loss: 1.4002 - val_accuracy: 0.5044
Epoch 

COMET INFO: [1.3889837265014648, 0.504444420337677]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/94ff600ec47f4a6faf2685367cd5b814
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [33]                : (0.39411306381225586, 0.6104454398155212)
COMET INFO:     batch_accuracy [1584]        : (0.125, 0.8125)
COMET INFO:     batch_loss [1584]            : (0.7375304698944092, 2.6997132301330566)
COMET INFO:     epoch_duration [33]          : (2.3910000000032596, 7.35899999999674)
COMET INFO:     loss [33]                    : (0.9985768795013428, 1.733643889427185)
COMET INFO:     val_accuracy [33]            : (0.3844444453716278, 0.5600000023841858)
COMET INFO:     val_loss [33]                : (1.315576195716858, 1.619643

Epoch 1/30
480/480 [==============================] - 7s 12ms/step - loss: 1.7990 - accuracy: 0.3967 - val_loss: 1.7092 - val_accuracy: 0.4311
Epoch 2/30
480/480 [==============================] - 5s 11ms/step - loss: 1.4704 - accuracy: 0.4575 - val_loss: 1.5348 - val_accuracy: 0.4778
Epoch 3/30
480/480 [==============================] - 5s 11ms/step - loss: 1.3730 - accuracy: 0.4932 - val_loss: 1.5214 - val_accuracy: 0.4556
Epoch 4/30
480/480 [==============================] - 3s 5ms/step - loss: 1.2862 - accuracy: 0.5120 - val_loss: 1.4260 - val_accuracy: 0.4978
Epoch 5/30
480/480 [==============================] - 3s 5ms/step - loss: 1.2613 - accuracy: 0.5322 - val_loss: 1.3659 - val_accuracy: 0.5133
Epoch 6/30
480/480 [==============================] - 3s 6ms/step - loss: 1.2090 - accuracy: 0.5402 - val_loss: 1.5193 - val_accuracy: 0.4844
Epoch 7/30
480/480 [==============================] - 3s 7ms/step - loss: 1.1767 - accuracy: 0.5547 - val_loss: 1.3259 - val_accuracy: 0.5289
Epo

COMET INFO: [1.3634496927261353, 0.5244444608688354]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/51cc732f39a041c3a8153883bb169f4f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [21]                : (0.39671790599823, 0.5851784348487854)
COMET INFO:     batch_accuracy [1008]        : (0.0, 0.75)
COMET INFO:     batch_loss [1008]            : (0.8131793737411499, 2.8120861053466797)
COMET INFO:     epoch_duration [21]          : (2.452999999994063, 7.188000000009197)
COMET INFO:     loss [21]                    : (1.0649036169052124, 1.7990400791168213)
COMET INFO:     val_accuracy [21]            : (0.41555556654930115, 0.5333333611488342)
COMET INFO:     val_loss [21]                : (1.3258519172668457, 1.709199905

In [11]:
# Build the model
model = Sequential()
model.add(
    Dense(
        units=450,
        activation='elu',
        input_shape=(num_train.shape[1],)
    )
)
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(
    Dense(
        units=256,
        activation='elu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model_meta = Sequential()
model_meta.add(
    Dense(
        units=900,
        activation='elu',
        input_shape=(num_train_meta.shape[1],)
    )
)
model_meta.add(Dropout(0.5))
model_meta.add(BatchNormalization())

model_meta.add(
    Dense(
        units=400,
        activation='elu'))
model_meta.add(Dropout(0.3))
model_meta.add(BatchNormalization())


merged_out = Concatenate()([model.output,model_meta.output])
merged_out = Dense(
    units=32,
    activation='elu')(merged_out)
merged_out = Dense(8,activation='softmax')(merged_out)

combined_model = Model([model.input] + [model_meta.input], [merged_out])
combined_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
        )

In [12]:
combined_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 dense_input (InputLayer)       [(None, 148)]        0           []                               
                                                                                                  
 dense_2_input (InputLayer)     [(None, 3)]          0           []                               
                                                                                                  
 dense (Dense)                  (None, 450)          67050       ['dense_input[0][0]']            
                                                                                                  
 dense_2 (Dense)                (None, 900)          3600        ['dense_2_input[0][0]']          
                                                                                              

In [22]:
combined_model.fit([num_train,num_train_meta],num_train_y,validation_data=([num_valid,num_valid_meta],num_valid_y),epochs=20,batch_size=32,shuffle=True,class_weight=class_weight,callbacks=[early_stop])

Epoch 1/20
240/240 [==============================] - 3s 8ms/step - loss: 1.7233 - accuracy: 0.4064 - val_loss: 1.5251 - val_accuracy: 0.4111
Epoch 2/20
240/240 [==============================] - 1s 5ms/step - loss: 1.4096 - accuracy: 0.4828 - val_loss: 1.3281 - val_accuracy: 0.5356
Epoch 3/20
240/240 [==============================] - 1s 6ms/step - loss: 1.3172 - accuracy: 0.5091 - val_loss: 1.4720 - val_accuracy: 0.4822
Epoch 4/20
240/240 [==============================] - 1s 6ms/step - loss: 1.2772 - accuracy: 0.5198 - val_loss: 1.3004 - val_accuracy: 0.5244
Epoch 5/20
240/240 [==============================] - 1s 5ms/step - loss: 1.2109 - accuracy: 0.5483 - val_loss: 1.3472 - val_accuracy: 0.5378
Epoch 6/20
240/240 [==============================] - 1s 6ms/step - loss: 1.1852 - accuracy: 0.5520 - val_loss: 1.3688 - val_accuracy: 0.5289
Epoch 7/20
240/240 [==============================] - 2s 9ms/step - loss: 1.1598 - accuracy: 0.5525 - val_loss: 1.2768 - val_accuracy: 0.5844
Epoch 

In [23]:
num_test_data = df_test_features.iloc[:,9:]
num_test_meta_data = df_test_features.iloc[:,3:5]
num_test_meta_data['duration'] = df_test_features['duration']

num_test = scaler.fit_transform(num_test_data)
num_test_meta = scaler.fit_transform(num_test_meta_data)

df_test_label = pd.read_csv('test_labels.csv')
num_test_y = pd.get_dummies(df_test_label['genre'])

In [24]:
from sklearn.metrics import classification_report
validation = combined_model.predict([num_valid,num_valid_meta])
validation = validation.argmax(axis=1)
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(validation)
correct_labels = df_valid_labels['genre'].values
print(classification_report(correct_labels,predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.46      0.65      0.54        55
dance and electronica       0.62      0.40      0.49        45
                 folk       0.45      0.56      0.50        64
       jazz and blues       0.55      0.50      0.52        44
                metal       0.82      0.68      0.74        66
                  pop       0.63      0.54      0.58        74
                 punk       0.46      0.55      0.50        44
      soul and reggae       0.79      0.72      0.76        58

             accuracy                           0.58       450
            macro avg       0.60      0.58      0.58       450
         weighted avg       0.61      0.58      0.59       450



In [25]:
combined_model.evaluate([num_test,num_test_meta],num_test_y)

14/14 [==============================] - 0s 2ms/step - loss: 1.7532 - accuracy: 0.4276


[1.753165602684021, 0.427570104598999]